In [8]:
import os
from pyairtable import Api
from pyairtable import api
from dotenv import load_dotenv
from fibsem_tools import read
from pyairtable.formulas import match
from cellmap_utils import upsert_image

load_dotenv()

at_api = Api(os.environ['AIRTABLE_API_KEY'])
# image table (switch between prod and test bases in .env)
image_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['IMAGE_TABLE_ID'])
collection_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['COLLECTION_TABLE_ID'])
fibsem_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['FIBSEM_TABLE_ID'])
annotation_table = at_api.table(os.environ['AIRTABLE_BASE_ID'], os.environ['ANNOTATION_TABLE_ID'])

In [9]:

for i in range(1, 5, 1):
    print(i)

    ds_name = f'jrc_mosquito-stylet-{i}'
    image_name = 'fibsem-uint8'
    image_path = f'/nrs/cellmap/data/jrc_mosquito-stylet-{i}/jrc_mosquito-stylet-{i}.zarr/recon-1/em/fibsem-uint8/'

    print(ds_name)
    print(image_path)
    upsert_image(image_table,
                 ds_name,
                 image_name,
                 image_path,
                 'em',
                 collection_table, 
                 fibsem_table,
                 annotation_table)

1
jrc_mosquito-stylet-1
/nrs/cellmap/data/jrc_mosquito-stylet-1/jrc_mosquito-stylet-1.zarr/recon-1/em/fibsem-uint8/


KeyError: 'multiscales'

In [ ]:
# ['/nrs/cellmap/data/jrc_mus-liver-4/jrc_mus-liver-4.zarr/recon-1/em/fibsem-uint8/',
#  '/nrs/cellmap/data/jrc_mus-liver-5/jrc_mus-liver-5.zarr/recon-1/em/fibsem-uint8/',
#  '/nrs/cellmap/data/jrc_mus-liver-6/jrc_mus-liver-6.zarr/recon-1/em/fibsem-uint8/',
#  '/nrs/cellmap/data/jrc_mus-liver-7/jrc_mus-liver-7.zarr/recon-1/em/fibsem-uint8/']
for i in range(4, 8, 1):
    print(i)

    ds_name = f'jrc_mus-liver-{i}'
    image_name = 'fibsem-uint8'
    image_path = f'/nrs/cellmap/data/{ds_name}/{ds_name}.zarr/recon-1/em/fibsem-uint8/s0'

    print(ds_name)
    print(image_path)
    upsert_image(image_table,
                 ds_name,
                 image_name,
                 image_path,
                 'em',
                 collection_table, 
                 fibsem_table,
                 annotation_table)

  Attempting uninstall: cellmap-utils
    Found existing installation: cellmap-utils 0.0.12
    Uninstalling cellmap-utils-0.0.12:
      Successfully uninstalled cellmap-utils-0.0.12


In [7]:
!pip install cellmap_utils==0.0.14

  Attempting uninstall: cellmap_utils
    Found existing installation: cellmap-utils 0.0.13
    Uninstalling cellmap-utils-0.0.13:
      Successfully uninstalled cellmap-utils-0.0.13


In [12]:
from fibsem_tools import read
import zarr
zg = read('/nrs/cellmap/data/jrc_mosquito-stylet-1/jrc_mosquito-stylet-1.zarr/recon-1/em/fibsem-uint8/')
if isinstance(zg, zarr.Group):
    print('group')
zg.attrs['multiscales']

group


[{'axes': [{'name': 'z', 'type': 'space', 'unit': 'nanometer'},
   {'name': 'y', 'type': 'space', 'unit': 'nanometer'},
   {'name': 'x', 'type': 'space', 'unit': 'nanometer'}],
  'coordinateTransformations': [{'scale': [1.0, 1.0, 1.0], 'type': 'scale'}],
  'datasets': [{'coordinateTransformations': [{'scale': [4.0, 4.0, 4.0],
      'type': 'scale'},
     {'translation': [0.0, 0.0, 0.0], 'type': 'translation'}],
    'path': 's0'},
   {'coordinateTransformations': [{'scale': [8.0, 8.0, 8.0], 'type': 'scale'},
     {'translation': [2.0, 2.0, 2.0], 'type': 'translation'}],
    'path': 's1'},
   {'coordinateTransformations': [{'scale': [16.0, 16.0, 16.0],
      'type': 'scale'},
     {'translation': [6.0, 6.0, 6.0], 'type': 'translation'}],
    'path': 's2'},
   {'coordinateTransformations': [{'scale': [32.0, 32.0, 32.0],
      'type': 'scale'},
     {'translation': [14.0, 14.0, 14.0], 'type': 'translation'}],
    'path': 's3'},
   {'coordinateTransformations': [{'scale': [64.0, 64.0, 64.0]

In [ ]:
from typing import Tuple
from pyairtable import api
from pyairtable.formulas import match
from fibsem_tools import read
import os
import zarr

# upsert image record
from typing import Literal
def upsert_image( image_table : api.table.Table,
                 ds_name : str,
                 image_name : str,
                 image_path : str,
                 image_type : Literal['human_segmentation', 'em'],
                 collection_table : api.table.Table,
                 fibsem_table : api.table.Table,
                 annotation_table : api.table.Table
                 ):
    
    existing_records = image_table.all(formula = match({'name' : image_name, 'location' : image_path.rstrip('/')}))
    
    if image_type == 'human_segmentation':
            value_type = 'label'
    else: 
        value_type = 'scalar'
    
    input_zarr = read(image_path.rstrip('/'))
    if isinstance(input_zarr, zarr.Group):
        zg = input_zarr
        z_arr_name = 's0'
    else:
        zg_path, z_arr_name = os.path.split(image_path.rstrip('/'))
        zg = read(zg_path)    
        
    
    scale = zg.attrs['multiscales'][0]['datasets'][0]['coordinateTransformations'][0]['scale']
    offset = zg.attrs['multiscales'][0]['datasets'][0]['coordinateTransformations'][1]['translation']
    shape  = zg[z_arr_name].shape
    
    try:
        fibsem_imaging = [fibsem_table.all(formula = match({'name' : ds_name}))[0]['id']]
    except:
        fibsem_imaging = []
     
    try:   
        annotation = [annotation_table.all(formula = match({'name' : image_name}))[0]['id']]
    except:
        annotation = []
    
    print(z_arr_name)
    record_to_upsert = {    'name' : image_name,
                            'collection' : [collection_table.all(formula = match({'id' : ds_name}))[0]['id']],
                            'location' : image_path.rstrip('/'),
                            'format' : 'zarr',
                            'image_type' : image_type,
                            'value_type' : value_type,
                            'size_x_pix' : shape[2],
                            'size_y_pix' : shape[1],
                            'size_z_pix' : shape[0],
                            'resolution_x_nm' : scale[2],
                            'resolution_y_nm' : scale[1],
                            'resolution_z_nm' : scale[0],
                            'offset_x_nm' : offset[2],
                            'offset_y_nm' : offset[1],
                            'offset_z_nm' : offset[0],
                            'fibsem_imaging' : fibsem_imaging,
                            'annotation' : annotation
                            }
    
    
    if len(existing_records) > 2:
        raise ValueError('Multiple records with matching input image name found')
    
    if not existing_records:
        image_table.create(record_to_upsert)
    elif len(existing_records) == 1:
        image_table.update(existing_records[0]['id'], record_to_upsert)
        
        


In [14]:

for i in range(1, 5, 1):
    print(i)

    ds_name = f'jrc_mosquito-stylet-{i}'
    image_name = 'fibsem-uint8'
    image_path = f'/nrs/cellmap/data/jrc_mosquito-stylet-{i}/jrc_mosquito-stylet-{i}.zarr/recon-1/em/fibsem-uint8/'

    print(ds_name)
    print(image_path)
    upsert_image(image_table,
                 ds_name,
                 image_name,
                 image_path,
                 'em',
                 collection_table, 
                 fibsem_table,
                 annotation_table)

1
jrc_mosquito-stylet-1
/nrs/cellmap/data/jrc_mosquito-stylet-1/jrc_mosquito-stylet-1.zarr/recon-1/em/fibsem-uint8/


UnboundLocalError: cannot access local variable 'zg_path' where it is not associated with a value